In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from capo.analysis.utils import (
    get_results,
    aggregate_results,
    get_prompt_scores,
    generate_comparison_table,
)
from capo.analysis.visualizations import (
    plot_population_scores,
    plot_population_members,
    plot_population_scores_comparison,
    plot_performance_profile_curve,
    plot_train_test_comparison,
    plot_length_score,
)

plot_population_scores(
    "gsm8k",
    "mistral",
    "CAPO",
    plot_seeds=True,
    plot_stddev=True,
    x_col="input_tokens_cum",
    color=sns.color_palette("Dark2")[0],
    add_title=True,
    add_legend=True,
)

In [ ]:
plot_performance_profile_curve(path_prefix="../temp/res/")

In [ ]:
plot_population_scores_comparison(
    "gsm8k",
    "mistral",
    optims=["CAPO", "EvoPromptGA", "OPRO", "PromptWizard"],
    plot_seeds=True,
    plot_stddev=True,
    x_col="input_tokens_cum",
);

In [ ]:
plot_train_test_comparison(
    "subj",
    "llama",
    optims=["CAPO", "EvoPromptGA", "OPRO"],
    plot_stddev=False,
    x_col="input_tokens_cum",
    path_prefix="../temp/res/",
);

In [ ]:
plot_population_members("gsm8k", "mistral", optim="CAPO", x_col="step", seeds=[44]);

In [ ]:
plot_length_score(
    "copa",
    "mistral",
    ["CAPO", "EvoPromptGA", "OPRO"],
    x_col="instr_len",
    score_col="test_score",
    log_scale=True,
);

In [ ]:
generate_comparison_table(model="mistral")

# Ablation Study

In [ ]:
ablation_runs = [
    "CAPO",
    "EvoPromptGA",
    "EvoPromptGA_TA",
    "CAPO_zero_shot",
    "CAPO_no_lp",
    "CAPO_no_racing",
]

plot_population_scores_comparison(
    "agnews", "llama", ablation_runs, plot_seeds=False, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
plot_population_scores_comparison(
    "gsm8k", "llama", ablation_runs, plot_seeds=False, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
generate_comparison_table(
    model="llama",
    cutoff_tokens=5_000_000,
    optims=ablation_runs,
    datasets=["agnews", "gsm8k"],
)

# HP search 

### gamma

In [ ]:
hp_runs = [
    "CAPO",
    "CAPO_gamma_0.01",
    "CAPO_gamma_0.05",
    "CAPO_gamma_0.1",
]

In [ ]:
plot_population_scores_comparison(
    "agnews", "llama", hp_runs, "mean", plot_seeds=False, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
plot_population_scores_comparison(
    "gsm8k", "llama", hp_runs, "mean", plot_seeds=False, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
generate_comparison_table(
    model="llama",
    cutoff_tokens=5_000_000,
    optims=hp_runs,
    datasets=["agnews", "gsm8k"],
)

### N_crossovers

In [ ]:
hp_runs = [
    "CAPO",
    "CAPO_ncrossovers_4",
    "CAPO_ncrossovers_7",
    "CAPO_ncrossovers_10",
]

In [ ]:
plot_population_scores_comparison(
    "agnews", "llama", hp_runs, "mean", plot_seeds=False, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
plot_population_scores_comparison(
    "gsm8k", "llama", hp_runs, "mean", plot_seeds=True, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
generate_comparison_table(
    model="llama",
    cutoff_tokens=5_000_000,
    optims=hp_runs,
    datasets=["agnews", "gsm8k"],
)

### Pop size

In [ ]:
hp_runs = ["CAPO", "CAPO_pop_6", "CAPO_pop_8", "CAPO_pop_10", "CAPO_pop_12"]

In [ ]:
plot_population_scores_comparison(
    "agnews", "llama", hp_runs, "mean", plot_seeds=True, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
plot_population_scores_comparison(
    "gsm8k", "llama", hp_runs, "mean", plot_seeds=True, plot_stddev=True, x_col="input_tokens_cum"
);

In [ ]:
generate_comparison_table(
    model="llama",
    cutoff_tokens=5_000_000,
    optims=hp_runs,
    datasets=["agnews", "gsm8k"],
)

# unverified plots! 👇🏻

In [ ]:
plot_population_members(DATASET, MODEL, OPTIM, x_col="cum_token");

In [ ]:
# def analyze_convergence_rate(
#     dataset, model, optims, agg="mean", score_col="test_score", convergence_threshold=0.9
# ):
#     """
#     Analyze how quickly each algorithm converges to its maximum performance.

#     Parameters:
#     -----------
#     convergence_threshold : float
#         The fraction of maximum performance used to define convergence
#         (e.g., 0.9 means 90% of the maximum score)
#     """
#     fig, ax = plt.subplots(figsize=(10, 6))

#     colors = ["blue", "red"]
#     results = []

#     for i, optim in enumerate(optims):
#         df = get_data(dataset, model, optim)
#         df = aggregate_population_scores(df, agg)

#         df_avg = df.groupby("step", as_index=False).max(numeric_only=True)

#         # Get the maximum score achieved by this optimizer
#         max_score = df_avg[score_col].max()
#         threshold_score = max_score * convergence_threshold

#         # Find when the algorithm first crosses the threshold
#         try:
#             convergence_point = df_avg[df_avg[score_col] >= threshold_score].iloc[0]
#             tokens_to_converge = convergence_point["cum_token"]
#             steps_to_converge = convergence_point["step"]

#             # Plot vertical line at convergence point
#             ax.axvline(x=tokens_to_converge, color=colors[i], linestyle="--", alpha=0.5)

#             # Store result
#             results.append(
#                 {
#                     "optimizer": optim,
#                     "dataset": dataset,
#                     "model": model,
#                     "tokens_to_converge": tokens_to_converge,
#                     "steps_to_converge": steps_to_converge,
#                     "max_score": max_score,
#                 }
#             )
#         except IndexError:
#             # Optimizer never reaches the threshold
#             print(f"{optim} never reaches {convergence_threshold*100}% of its maximum performance")
#             results.append(
#                 {
#                     "optimizer": optim,
#                     "dataset": dataset,
#                     "model": model,
#                     "tokens_to_converge": float("inf"),
#                     "steps_to_converge": float("inf"),
#                     "max_score": max_score,
#                 }
#             )

#         # Plot performance curve
#         ax.plot(
#             df_avg["cum_token"],
#             df_avg[score_col],
#             label=f"{optim} (max: {max_score:.3f})",
#             color=colors[i],
#             linewidth=2,
#         )

#     ax.set_xlabel("Cumulative Tokens")
#     ax.set_ylabel(score_col)
#     ax.set_title(
#         f"{dataset} - {model} - Convergence Analysis ({convergence_threshold*100}% threshold)"
#     )
#     ax.legend()

#     # Create a DataFrame with the convergence results
#     results_df = pd.DataFrame(results)

#     return fig, results_df


# def plot_convergence_comparison(
#     datasets=["sst-5", "agnews", "rte", "gsm8k", "subj"],
#     optimizers=["CAPO", "EvoPromptGA"],
#     model="llama",
# ):
#     """
#     Create a bar chart comparing tokens to convergence across datasets.
#     """
#     all_results = []

#     for dataset in datasets:
#         _, results_df = analyze_convergence_rate(dataset, model, optimizers)
#         all_results.append(results_df)

#     combined_results = pd.concat(all_results)

#     # Create grouped bar chart
#     fig, ax = plt.subplots(figsize=(12, 6))

#     # Get unique datasets and optimizers
#     unique_datasets = combined_results["dataset"].unique()
#     unique_optims = combined_results["optimizer"].unique()

#     x = np.arange(len(unique_datasets))
#     width = 0.35

#     # Create bars for each optimizer
#     for i, optim in enumerate(unique_optims):
#         optim_data = combined_results[combined_results["optimizer"] == optim]

#         # Convert 'inf' to a large but finite number for plotting
#         tokens_data = [
#             optim_data[optim_data["dataset"] == dataset]["tokens_to_converge"].values[0]
#             if dataset in optim_data["dataset"].values
#             and optim_data[optim_data["dataset"] == dataset]["tokens_to_converge"].values[0]
#             != float("inf")
#             else np.nan
#             for dataset in unique_datasets
#         ]

#         # Plot bars
#         ax.bar(x + (i - 0.5) * width, tokens_data, width, label=optim)

#     # Customize the plot
#     ax.set_xlabel("Dataset")
#     ax.set_ylabel("Tokens to Convergence")
#     ax.set_title(f"Tokens Required to Reach 90% of Maximum Performance - {model}")
#     ax.set_xticks(x)
#     ax.set_xticklabels(unique_datasets)
#     ax.legend()

#     return fig


# import numpy as np

# # Example usage
# _, results_df = analyze_convergence_rate(
#     "gsm8k", "llama", ["CAPO", "EvoPromptGA"], convergence_threshold=0.99
# )
# plot_convergence_comparison();

In [ ]:
# def plot_compute_efficiency(
#     dataset, model, optims, cost_per_1k_tokens=0.005, score_col="test_score"
# ):
#     """
#     Analyze the cost-effectiveness of each algorithm.

#     This assumes a simple cost model based on token usage.
#     Adapt the cost_per_1k_tokens parameter based on your model costs.
#     """
#     fig, ax = plt.subplots(figsize=(10, 6))

#     colors = ["blue", "red"]

#     for i, optim in enumerate(optims):
#         df = get_data(dataset, model, optim)
#         df = aggregate_population_scores(df, "mean")

#         df_avg = df.groupby("step", as_index=False).mean(numeric_only=True)

#         # Calculate cost
#         df_avg["cost"] = df_avg["cum_token"] * cost_per_1k_tokens / 1000

#         # Calculate cost effectiveness as score per dollar
#         df_avg["cost_effectiveness"] = df_avg[score_col] / df_avg["cost"]

#         # Plot cost effectiveness
#         ax.plot(
#             df_avg["cum_token"],
#             df_avg["cost_effectiveness"],
#             label=f"{optim}",
#             color=colors[i],
#             linewidth=2,
#         )

#     ax.set_xlabel("Cumulative Tokens")
#     ax.set_ylabel("Score per Dollar")
#     ax.set_title(f"{dataset} - {model} - Cost Effectiveness")
#     ax.legend()

#     return fig


# def plot_cost_performance_tradeoff(
#     datasets=["sst-5", "agnews", "rte", "gsm8k", "subj"],
#     optimizers=["CAPO", "EvoPromptGA"],
#     model="llama",
#     cutoff_tokens=5_000_000,
#     cost_per_1k_tokens=0.005,
#     agg="mean",
# ):
#     """
#     Create a scatter plot showing the cost-performance tradeoff across datasets.
#     """
#     fig, ax = plt.subplots(figsize=(10, 8))

#     # Define markers for datasets
#     markers = ["o", "s", "^", "D", "p"]  # circle, square, triangle, diamond, pentagon

#     # Store results for annotation
#     results = []

#     # Loop through optimizers and datasets
#     for i, optim in enumerate(optimizers):
#         costs = []
#         scores = []
#         dataset_labels = []

#         for j, dataset in enumerate(datasets):
#             try:
#                 df = get_data(dataset, model, optim)
#                 df = aggregate_population_scores(df, agg)

#                 # Get values at token cutoff
#                 max_steps = []
#                 for seed in df.seed.unique():
#                     df_seed = df[df.seed == seed]
#                     last_step = df_seed.loc[df_seed["cum_token"] < cutoff_tokens, "step"].max()
#                     if pd.isna(last_step):
#                         continue
#                     max_steps.append(df_seed[df_seed["step"] == last_step])

#                 if max_steps:
#                     final_df = pd.concat(max_steps)
#                     avg_score = final_df["test_score"].mean() * 100  # Convert to percentage
#                     avg_tokens = final_df["cum_token"].mean()
#                     cost = avg_tokens * cost_per_1k_tokens / 1000  # Cost in dollars

#                     costs.append(cost)
#                     scores.append(avg_score)
#                     dataset_labels.append(dataset)

#                     # Store result
#                     results.append(
#                         {"dataset": dataset, "optimizer": optim, "cost": cost, "score": avg_score}
#                     )
#             except Exception as e:
#                 print(f"Error processing {dataset} - {optim}: {e}")

#         # Plot scatter points
#         for j, (cost, score, dataset) in enumerate(zip(costs, scores, dataset_labels)):
#             ax.scatter(
#                 cost,
#                 score,
#                 marker=markers[j % len(markers)],
#                 s=100,
#                 color=f"C{i}",
#                 label=f"{optim} - {dataset}",
#             )

#     # Add labels
#     ax.set_xlabel("Estimated Cost ($)")
#     ax.set_ylabel("Test Score (%)")
#     ax.set_title(f"Cost-Performance Tradeoff - {model}")

#     # Add legend outside the plot
#     ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

#     # Adjust layout to make room for the legend
#     plt.tight_layout(rect=[0, 0, 0.85, 1])

#     # Create results dataframe
#     results_df = pd.DataFrame(results)

#     return fig, results_df


# # Example usage
# plot_compute_efficiency("gsm8k", "llama", ["CAPO", "EvoPromptGA"])
# plot_cost_performance_tradeoff();

In [ ]:
# def fit_learning_curve(dataset, model, optims, score_col="test_score"):
#     """
#     Fit a learning curve model to estimate asymptotic performance.
#     Uses an exponential approach model: s(t) = a - b*exp(-c*t)
#     where:
#     - a is the asymptotic performance
#     - b is the scaling factor
#     - c is the learning rate
#     - t is the number of tokens
#     """
#     from scipy.optimize import curve_fit

#     fig, ax = plt.subplots(figsize=(10, 6))

#     colors = ["blue", "red"]
#     results = []

#     # Define learning curve function
#     def learning_curve(x, a, b, c):
#         return a - b * np.exp(-c * x)

#     for i, optim in enumerate(optims):
#         df = get_data(dataset, model, optim)
#         df = aggregate_population_scores(df, "mean")

#         df_avg = df.groupby("step", as_index=False).mean(numeric_only=True)

#         # Prepare data for fitting
#         x_data = df_avg["cum_token"].values
#         y_data = df_avg[score_col].values

#         try:
#             # Initial parameter guess
#             p0 = [max(y_data) * 1.1, max(y_data) - min(y_data), 1e-6]

#             # Fit curve
#             popt, pcov = curve_fit(learning_curve, x_data, y_data, p0=p0, maxfev=10000)

#             # Extract parameters
#             a, b, c = popt

#             # Calculate extrapolated performance
#             x_extrapolated = np.linspace(min(x_data), max(x_data) * 2, 100)
#             y_extrapolated = learning_curve(x_extrapolated, a, b, c)

#             # Plot actual data
#             ax.plot(x_data, y_data, "o", color=colors[i], alpha=0.5, label=f"{optim} (actual)")

#             # Plot fitted curve
#             ax.plot(
#                 x_extrapolated,
#                 y_extrapolated,
#                 "-",
#                 color=colors[i],
#                 label=f"{optim} (fitted, asymptote={a:.3f})",
#             )

#             # Store parameters
#             results.append(
#                 {
#                     "dataset": dataset,
#                     "model": model,
#                     "optimizer": optim,
#                     "asymptotic_performance": a,
#                     "scaling_factor": b,
#                     "learning_rate": c,
#                     "tokens_to_90pct": -np.log(0.1) / c if c > 0 else float("inf"),
#                     "tokens_to_95pct": -np.log(0.05) / c if c > 0 else float("inf"),
#                     "tokens_to_99pct": -np.log(0.01) / c if c > 0 else float("inf"),
#                 }
#             )

#         except Exception as e:
#             print(f"Error fitting curve for {optim}: {e}")

#     ax.set_xlabel("Cumulative Tokens")
#     ax.set_ylabel(score_col)
#     ax.set_title(f"{dataset} - {model} - Learning Curve Extrapolation")
#     ax.legend()

#     # Create results dataframe
#     results_df = pd.DataFrame(results)

#     return fig, results_df


# def plot_asymptotic_comparison(
#     datasets=["sst-5", "agnews", "rte", "gsm8k", "subj"],
#     optimizers=["CAPO", "EvoPromptGA"],
#     model="llama",
# ):
#     """
#     Create a bar chart comparing the estimated asymptotic performance across datasets.
#     """
#     fig, ax = plt.subplots(figsize=(12, 6))

#     # Store all results
#     all_results = []

#     for dataset in datasets:
#         try:
#             _, results_df = fit_learning_curve(dataset, model, optimizers)
#             all_results.append(results_df)
#         except Exception as e:
#             print(f"Error processing {dataset}: {e}")

#     # Combine results
#     if all_results:
#         combined_results = pd.concat(all_results)

#         # Set up grouped bar chart
#         unique_datasets = combined_results["dataset"].unique()
#         unique_optims = combined_results["optimizer"].unique()

#         x = np.arange(len(unique_datasets))
#         width = 0.35

#         # Create bars for each optimizer
#         for i, optim in enumerate(unique_optims):
#             optim_data = combined_results[combined_results["optimizer"] == optim]

#             asymptotic_data = [
#                 optim_data[optim_data["dataset"] == dataset]["asymptotic_performance"].values[0]
#                 * 100
#                 if dataset in optim_data["dataset"].values
#                 else np.nan
#                 for dataset in unique_datasets
#             ]

#             # Plot bars
#             ax.bar(x + (i - 0.5) * width, asymptotic_data, width, label=optim)

#         # Customize plot
#         ax.set_xlabel("Dataset")
#         ax.set_ylabel("Asymptotic Performance (%)")
#         ax.set_title(f"Predicted Maximum Performance - {model}")
#         ax.set_xticks(x)
#         ax.set_xticklabels(unique_datasets)
#         ax.legend()
#     else:
#         ax.text(0.5, 0.5, "No results available", ha="center", va="center", transform=ax.transAxes)

#     return fig


# # Example usage
# fit_learning_curve("gsm8k", "llama", ["CAPO", "EvoPromptGA"])
# plot_asymptotic_comparison()

In [ ]:
# def analyze_prompt_diversity(dataset, model, optimizers, seed=None, max_steps=None):
#     """
#     Analyze the diversity of prompts generated during optimization.

#     Diversity can be measured in multiple ways:
#     1. Lexical diversity (using metrics like TTR - Type-Token Ratio)
#     2. Semantic diversity (using embeddings and clustering)
#     3. Length distribution
#     4. Instruction pattern changes

#     Parameters:
#     -----------
#     dataset : str
#         The dataset name
#     model : str
#         The model name
#     optimizers : list
#         List of optimizer names to compare
#     seed : int, optional
#         If provided, analyze only this seed. Otherwise, analyze all seeds.
#     max_steps : int, optional
#         If provided, limit analysis to this many steps
#     """
#     import re
#     import numpy as np
#     from collections import Counter
#     import matplotlib.pyplot as plt
#     from sklearn.feature_extraction.text import TfidfVectorizer
#     from sklearn.metrics.pairwise import cosine_similarity

#     results = []

#     for optimizer in optimizers:
#         # Get data using the existing function
#         df = get_data(dataset, model, optimizer)

#         # Filter for specific seed if requested
#         if seed is not None:
#             df = df[df.seed == seed]

#         if df.empty:
#             print(f"No data found for {optimizer}")
#             continue

#         # Check if 'prompt' column exists
#         if "prompt" not in df.columns:
#             print(f"No 'prompt' column found for {optimizer}")
#             continue

#         # Group by seed and step to ensure we have one prompt per step
#         df_prompts = df.groupby(["seed", "step"]).first().reset_index()

#         # Limit steps if specified
#         if max_steps is not None and max_steps < len(df_prompts):
#             step_indices = list(range(0, len(df_prompts), len(df_prompts) // max_steps))
#             df_prompts = df_prompts.iloc[step_indices]

#         # Extract prompts, steps, and seeds
#         prompts = df_prompts["prompt"].tolist()
#         steps = df_prompts["step"].tolist()
#         seeds = df_prompts["seed"].tolist()

#         if not prompts:
#             print(f"No prompts found for {optimizer}")
#             continue

#         # Calculate lexical diversity (TTR) for each prompt
#         def calculate_ttr(text):
#             tokens = re.findall(r"\b\w+\b", text.lower())
#             if not tokens:
#                 return 0
#             return len(set(tokens)) / len(tokens)

#         ttrs = [calculate_ttr(p) for p in prompts]

#         # Calculate prompt lengths
#         lengths = [len(p) for p in prompts]

#         # Calculate similarity matrix using TF-IDF
#         vectorizer = TfidfVectorizer()
#         try:
#             tfidf_matrix = vectorizer.fit_transform(prompts)
#             similarity_matrix = cosine_similarity(tfidf_matrix)

#             # Calculate average similarity to previous prompt
#             prev_similarities = []
#             for i in range(1, len(prompts)):
#                 prev_similarities.append(similarity_matrix[i, i - 1])

#             # Calculate average similarity to all other prompts
#             avg_similarities = []
#             for i in range(len(prompts)):
#                 # Exclude self-similarity (which is always 1.0)
#                 other_similarities = [
#                     similarity_matrix[i, j] for j in range(len(prompts)) if j != i
#                 ]
#                 avg_similarities.append(np.mean(other_similarities))
#         except Exception as e:
#             print(f"Error calculating similarities for {optimizer}: {e}")
#             prev_similarities = []
#             avg_similarities = [0] * len(prompts)

#         # Store results
#         for i, (step, ttr, length, avg_sim) in enumerate(
#             zip(steps, ttrs, lengths, avg_similarities)
#         ):
#             prev_sim = prev_similarities[i - 1] if i > 0 else None

#             results.append(
#                 {
#                     "dataset": dataset,
#                     "model": model,
#                     "optimizer": optimizer,
#                     "step": step,
#                     "ttr": ttr,
#                     "length": length,
#                     "avg_similarity": avg_sim,
#                     "prev_similarity": prev_sim,
#                 }
#             )

#     # Convert to DataFrame
#     results_df = pd.DataFrame(results)

#     # Create visualizations
#     if not results_df.empty:
#         # Create figure with subplots
#         fig, axes = plt.subplots(2, 2, figsize=(12, 10))

#         # Plot 1: Lexical diversity over time
#         for optimizer in optimizers:
#             optim_data = results_df[results_df["optimizer"] == optimizer]
#             if not optim_data.empty:
#                 axes[0, 0].plot(optim_data["step"], optim_data["ttr"], marker="o", label=optimizer)

#         axes[0, 0].set_xlabel("Step")
#         axes[0, 0].set_ylabel("Type-Token Ratio")
#         axes[0, 0].set_title("Lexical Diversity Over Time")
#         axes[0, 0].legend()

#         # Plot 2: Prompt length over time
#         for optimizer in optimizers:
#             optim_data = results_df[results_df["optimizer"] == optimizer]
#             if not optim_data.empty:
#                 axes[0, 1].plot(
#                     optim_data["step"], optim_data["length"], marker="o", label=optimizer
#                 )

#         axes[0, 1].set_xlabel("Step")
#         axes[0, 1].set_ylabel("Prompt Length (chars)")
#         axes[0, 1].set_title("Prompt Length Over Time")
#         axes[0, 1].legend()

#         # Plot 3: Average similarity to all prompts
#         for optimizer in optimizers:
#             optim_data = results_df[results_df["optimizer"] == optimizer]
#             if not optim_data.empty:
#                 axes[1, 0].plot(
#                     optim_data["step"], optim_data["avg_similarity"], marker="o", label=optimizer
#                 )

#         axes[1, 0].set_xlabel("Step")
#         axes[1, 0].set_ylabel("Average Similarity")
#         axes[1, 0].set_title("Prompt Similarity (to all prompts)")
#         axes[1, 0].legend()

#         # Plot 4: Similarity to previous prompt
#         for optimizer in optimizers:
#             optim_data = results_df[results_df["optimizer"] == optimizer].dropna(
#                 subset=["prev_similarity"]
#             )
#             if not optim_data.empty:
#                 axes[1, 1].plot(
#                     optim_data["step"], optim_data["prev_similarity"], marker="o", label=optimizer
#                 )

#         axes[1, 1].set_xlabel("Step")
#         axes[1, 1].set_ylabel("Similarity to Previous")
#         axes[1, 1].set_title("Sequential Prompt Changes")
#         axes[1, 1].legend()

#         plt.tight_layout()
#         plt.suptitle(f"Prompt Diversity Analysis: {dataset} - {model}", y=1.02)

#         return fig, results_df
#     else:
#         print("No data available for diversity analysis")
#         return None, None


# def visualize_prompt_space_evolution(
#     dataset, model, optimizer, seed=42, embedding_method="tfidf", dim_reduction="tsne"
# ):
#     """
#     Visualize how prompts evolve in the semantic space using dimensionality reduction.

#     Args:
#         dataset: The dataset name
#         model: The model name
#         optimizer: The optimizer name
#         seed: The seed to analyze
#         embedding_method: 'tfidf' or 'bert' for embedding calculation
#         dim_reduction: 'tsne' or 'pca' for dimensionality reduction
#     """
#     import numpy as np
#     import matplotlib.pyplot as plt
#     from sklearn.feature_extraction.text import TfidfVectorizer
#     from sklearn.decomposition import PCA
#     from sklearn.manifold import TSNE

#     # Get data using the existing function
#     df = get_data(dataset, model, optimizer)

#     # Filter for the specific seed
#     df = df[df.seed == seed]

#     if df.empty:
#         print(f"No data found for {optimizer} with seed {seed}")
#         return None

#     # Check if 'prompt' column exists
#     if "prompt" not in df.columns:
#         print(f"No 'prompt' column found for {optimizer}")
#         return None

#     # Group by step to ensure we have one prompt per step
#     df_prompts = df.groupby("step").first().reset_index()

#     # Sort by step
#     df_prompts = df_prompts.sort_values("step")

#     # Extract prompts and steps
#     prompts = df_prompts["prompt"].tolist()
#     steps = df_prompts["step"].tolist()

#     if not prompts:
#         print(f"No prompts found for {optimizer} with seed {seed}")
#         return None

#     # Create embeddings
#     if embedding_method == "tfidf":
#         vectorizer = TfidfVectorizer(max_features=100)
#         embeddings = vectorizer.fit_transform(prompts).toarray()
#     elif embedding_method == "bert":
#         # You would need a sentence transformer library for this
#         # For example with sentence-transformers:
#         # from sentence_transformers import SentenceTransformer
#         # model = SentenceTransformer('all-MiniLM-L6-v2')
#         # embeddings = model.encode(prompts)
#         print("BERT embeddings not implemented in this function")
#         return None

#     # Apply dimensionality reduction
#     if embeddings.shape[0] < 3:
#         print("Too few samples for meaningful dimensionality reduction")
#         return None

#     if dim_reduction == "tsne":
#         reducer = TSNE(n_components=2, random_state=42)
#     elif dim_reduction == "pca":
#         reducer = PCA(n_components=2, random_state=42)

#     reduced_embeddings = reducer.fit_transform(embeddings)

#     # Create visualization
#     fig, ax = plt.subplots(figsize=(10, 8))

#     # Create colormap for steps
#     step_colors = plt.cm.viridis(np.linspace(0, 1, len(steps)))

#     # Plot points
#     scatter = ax.scatter(
#         reduced_embeddings[:, 0],
#         reduced_embeddings[:, 1],
#         c=steps,
#         cmap="viridis",
#         s=100,
#         alpha=0.7,
#     )

#     # Add step labels
#     for i, (x, y, step) in enumerate(
#         zip(reduced_embeddings[:, 0], reduced_embeddings[:, 1], steps)
#     ):
#         ax.annotate(f"{step}", (x, y), xytext=(5, 5), textcoords="offset points")

#     # Connect points with lines to show evolution
#     ax.plot(reduced_embeddings[:, 0], reduced_embeddings[:, 1], "-", color="gray", alpha=0.5)

#     # Add colorbar
#     cbar = plt.colorbar(scatter)
#     cbar.set_label("Step")

#     # Customize the plot
#     ax.set_title(f"Prompt Space Evolution: {dataset} - {model} - {optimizer}")
#     ax.set_xlabel(f"{dim_reduction.upper()} Dimension 1")
#     ax.set_ylabel(f"{dim_reduction.upper()} Dimension 2")

#     return fig

In [ ]:
# # run
# analyze_prompt_diversity("gsm8k", "llama", ["CAPO", "EvoPromptGA"], max_steps=100)

In [ ]:
# def plot_optimization_trajectory(dataset, model, optimizers, seed=42, score_col="test_score"):
#     """
#     Visualize the optimization trajectory as a 2D or 3D path.

#     This shows how the optimization navigates through the performance landscape,
#     revealing exploration patterns, local optima, and search strategies.
#     """
#     import matplotlib.pyplot as plt
#     from mpl_toolkits.mplot3d import Axes3D
#     import numpy as np

#     # Create figures
#     fig1, ax1 = plt.subplots(figsize=(10, 6))  # Score vs. Step
#     fig2, ax2 = plt.subplots(figsize=(10, 6))  # Delta Score vs. Current Score
#     fig3 = plt.figure(figsize=(12, 10))
#     ax3 = fig3.add_subplot(111, projection="3d")  # 3D trajectory

#     colors = ["blue", "red", "green", "purple", "orange"]

#     for i, optimizer in enumerate(optimizers):
#         # Get the data
#         df = get_data(dataset, model, optimizer)

#         # Filter for the specific seed
#         df_seed = df[df.seed == seed]

#         if df_seed.empty:
#             print(f"No data for seed {seed} with optimizer {optimizer}")
#             continue

#         # Sort by step
#         df_seed = df_seed.sort_values("step")

#         # Extract key metrics
#         steps = df_seed["step"].values
#         scores = df_seed[score_col].values
#         tokens = df_seed["cum_token"].values

#         # Calculate score changes (deltas)
#         score_deltas = np.diff(scores, prepend=scores[0])

#         # Plot 1: Score vs Step (basic learning curve)
#         ax1.plot(steps, scores, marker="o", color=colors[i % len(colors)], label=optimizer)

#         # Plot 2: Delta Score vs Current Score (phase space)
#         ax2.scatter(
#             scores[:-1], score_deltas[1:], color=colors[i % len(colors)], label=optimizer, alpha=0.7
#         )
#         ax2.plot(scores[:-1], score_deltas[1:], color=colors[i % len(colors)], alpha=0.4)

#         # Add arrows to show direction
#         for j in range(0, len(scores) - 1, max(1, len(scores) // 10)):  # Add arrows at intervals
#             ax2.annotate(
#                 "",
#                 xy=(
#                     scores[j + 1],
#                     score_deltas[j + 2] if j + 2 < len(score_deltas) else score_deltas[j + 1],
#                 ),
#                 xytext=(scores[j], score_deltas[j + 1]),
#                 arrowprops=dict(arrowstyle="->", color=colors[i % len(colors)], lw=1.5),
#             )

#         # Plot 3: 3D trajectory (step, score, delta)
#         ax3.plot(
#             steps[:-1],
#             scores[:-1],
#             score_deltas[1:],
#             color=colors[i % len(colors)],
#             label=optimizer,
#         )
#         ax3.scatter(
#             steps[:-1], scores[:-1], score_deltas[1:], color=colors[i % len(colors)], alpha=0.7
#         )

#     # Customize the plots
#     ax1.set_xlabel("Step")
#     ax1.set_ylabel(score_col)
#     ax1.set_title(f"Learning Curve: {dataset} - {model} - Seed {seed}")
#     ax1.legend()
#     ax1.grid(True, linestyle="--", alpha=0.7)

#     ax2.set_xlabel("Score")
#     ax2.set_ylabel("Score Delta (Change)")
#     ax2.set_title(f"Phase Space: {dataset} - {model} - Seed {seed}")
#     ax2.legend()
#     ax2.grid(True, linestyle="--", alpha=0.7)
#     ax2.axhline(y=0, color="k", linestyle="-", alpha=0.3)  # Zero line

#     ax3.set_xlabel("Step")
#     ax3.set_ylabel("Score")
#     ax3.set_zlabel("Score Delta")
#     ax3.set_title(f"3D Trajectory: {dataset} - {model} - Seed {seed}")
#     ax3.legend()

#     return [fig1, fig2, fig3]


# def analyze_search_strategy(dataset, model, optimizer, seed=42, score_col="test_score"):
#     """
#     Analyze the search strategy by examining patterns in score changes.

#     This reveals insights about:
#     - Exploitation vs. exploration balance
#     - Local optima navigation
#     - Learning rate adaptation
#     """
#     import matplotlib.pyplot as plt
#     import numpy as np
#     from scipy.signal import find_peaks

#     # Get the data
#     df = get_data(dataset, model, optimizer)

#     # Filter for the specific seed
#     df_seed = df[df.seed == seed]

#     if df_seed.empty:
#         print(f"No data for seed {seed} with optimizer {optimizer}")
#         return None

#     # Sort by step
#     df_seed = df_seed.sort_values("step")

#     # Extract key metrics
#     steps = df_seed["step"].values
#     scores = df_seed[score_col].values
#     tokens = df_seed["cum_token"].values

#     # Calculate score changes
#     deltas = np.diff(scores, prepend=scores[0])
#     deltas = deltas[1:]  # Remove the first entry (which is just the initial score)

#     # Analyze sign changes (direction changes in optimization)
#     sign_changes = np.diff(np.signbit(deltas)).sum()

#     # Identify peaks (local maxima)
#     peaks, _ = find_peaks(scores)

#     # Identify valleys (local minima)
#     valleys, _ = find_peaks(-scores)

#     # Create figure with multiple subplots
#     fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

#     # Plot 1: Score over steps with peaks and valleys
#     axes[0].plot(steps, scores, marker="o", color="blue")
#     axes[0].scatter(
#         steps[peaks], scores[peaks], color="green", s=100, marker="^", label="Local Maxima"
#     )
#     axes[0].scatter(
#         steps[valleys], scores[valleys], color="red", s=100, marker="v", label="Local Minima"
#     )
#     axes[0].set_ylabel(score_col)
#     axes[0].set_title(
#         f"Optimization Landscape Navigation: {dataset} - {model} - {optimizer} - Seed {seed}"
#     )
#     axes[0].legend()
#     axes[0].grid(True, linestyle="--", alpha=0.7)

#     # Plot 2: Score deltas (first derivative)
#     axes[1].plot(steps[1:], deltas, marker="o", color="purple")
#     axes[1].axhline(y=0, color="k", linestyle="-", alpha=0.3)
#     axes[1].set_ylabel("Score Delta")
#     axes[1].set_title("Change in Score (First Derivative)")
#     axes[1].grid(True, linestyle="--", alpha=0.7)

#     # Plot 3: Second derivative (acceleration/deceleration in learning)
#     second_deriv = np.diff(deltas, prepend=deltas[0])
#     axes[2].plot(steps[1:], second_deriv, marker="o", color="orange")
#     axes[2].axhline(y=0, color="k", linestyle="-", alpha=0.3)
#     axes[2].set_ylabel("Score Acceleration")
#     axes[2].set_xlabel("Step")
#     axes[2].set_title("Acceleration in Learning (Second Derivative)")
#     axes[2].grid(True, linestyle="--", alpha=0.7)

#     # Add annotations with statistics
#     plt.figtext(
#         0.02,
#         0.02,
#         f"Search Analysis:\n"
#         f"- Direction changes: {sign_changes}\n"
#         f"- Local maxima: {len(peaks)}\n"
#         f"- Local minima: {len(valleys)}\n"
#         f"- Max improvement in single step: {deltas.max():.4f}\n"
#         f"- Max decline in single step: {deltas.min():.4f}\n"
#         f"- Avg positive step: {deltas[deltas > 0].mean() if any(deltas > 0) else 0:.4f}\n"
#         f"- Avg negative step: {deltas[deltas < 0].mean() if any(deltas < 0) else 0:.4f}",
#         fontsize=12,
#         bbox=dict(facecolor="white", alpha=0.8),
#     )

#     plt.tight_layout()
#     plt.subplots_adjust(bottom=0.15)  # Make room for annotations

#     return fig


# def compare_optimization_strategies(dataset, model, optimizers, seeds=None, score_col="test_score"):
#     """
#     Compare optimization strategies across algorithms and seeds by analyzing
#     the distribution of improvement jumps and exploration patterns.
#     """
#     import matplotlib.pyplot as plt
#     import numpy as np
#     import seaborn as sns

#     if seeds is None:
#         # Default to seeds 42, 43, 44 if no seeds specified
#         seeds = [42, 43, 44]

#     # Store all delta distributions
#     all_deltas = {optimizer: [] for optimizer in optimizers}

#     # Collect data for all optimizers and seeds
#     for optimizer in optimizers:
#         for seed in seeds:
#             # Get the data
#             df = get_data(dataset, model, optimizer)

#             # Filter for the specific seed
#             df_seed = df[df.seed == seed]

#             if df_seed.empty:
#                 print(f"No data for seed {seed} with optimizer {optimizer}")
#                 continue

#             # Sort by step
#             df_seed = df_seed.sort_values("step")

#             # Extract scores
#             scores = df_seed[score_col].values

#             # Calculate score deltas
#             deltas = np.diff(scores)

#             # Add to collection
#             all_deltas[optimizer].extend(deltas)

#     # Create figure with subplots
#     fig, axes = plt.subplots(2, 2, figsize=(15, 12))

#     # Plot 1: Distribution of score deltas (violin plot)
#     delta_data = []
#     for optimizer, deltas in all_deltas.items():
#         for delta in deltas:
#             delta_data.append({"optimizer": optimizer, "delta": delta})

#     delta_df = pd.DataFrame(delta_data)
#     sns.violinplot(x="optimizer", y="delta", data=delta_df, ax=axes[0, 0])
#     axes[0, 0].set_title("Distribution of Score Changes")
#     axes[0, 0].set_xlabel("Optimizer")
#     axes[0, 0].set_ylabel("Score Delta")
#     axes[0, 0].axhline(y=0, color="k", linestyle="--", alpha=0.7)

#     # Plot 2: Positive vs negative changes
#     pos_neg_data = []
#     for optimizer, deltas in all_deltas.items():
#         deltas = np.array(deltas)
#         pos_pct = np.sum(deltas > 0) / len(deltas) * 100 if len(deltas) > 0 else 0
#         neg_pct = np.sum(deltas < 0) / len(deltas) * 100 if len(deltas) > 0 else 0
#         zero_pct = np.sum(deltas == 0) / len(deltas) * 100 if len(deltas) > 0 else 0

#         pos_neg_data.append({"optimizer": optimizer, "type": "Improvement", "percentage": pos_pct})
#         pos_neg_data.append({"optimizer": optimizer, "type": "Decline", "percentage": neg_pct})
#         pos_neg_data.append({"optimizer": optimizer, "type": "No Change", "percentage": zero_pct})

#     pos_neg_df = pd.DataFrame(pos_neg_data)
#     sns.barplot(x="optimizer", y="percentage", hue="type", data=pos_neg_df, ax=axes[0, 1])
#     axes[0, 1].set_title("Direction of Changes")
#     axes[0, 1].set_xlabel("Optimizer")
#     axes[0, 1].set_ylabel("Percentage")
#     axes[0, 1].legend(title="")

#     # Plot 3: Improvement magnitudes
#     improvement_data = []
#     for optimizer, deltas in all_deltas.items():
#         deltas = np.array(deltas)

#         # Get positive deltas only
#         pos_deltas = deltas[deltas > 0]

#         if len(pos_deltas) > 0:
#             small_pct = np.sum(pos_deltas < 0.01) / len(pos_deltas) * 100
#             medium_pct = np.sum((pos_deltas >= 0.01) & (pos_deltas < 0.05)) / len(pos_deltas) * 100
#             large_pct = np.sum(pos_deltas >= 0.05) / len(pos_deltas) * 100

#             improvement_data.append(
#                 {"optimizer": optimizer, "type": "Small (<1%)", "percentage": small_pct}
#             )
#             improvement_data.append(
#                 {"optimizer": optimizer, "type": "Medium (1-5%)", "percentage": medium_pct}
#             )
#             improvement_data.append(
#                 {"optimizer": optimizer, "type": "Large (>5%)", "percentage": large_pct}
#             )

#     improvement_df = pd.DataFrame(improvement_data)
#     sns.barplot(x="optimizer", y="percentage", hue="type", data=improvement_df, ax=axes[1, 0])
#     axes[1, 0].set_title("Magnitude of Improvements")
#     axes[1, 0].set_xlabel("Optimizer")
#     axes[1, 0].set_ylabel("Percentage")
#     axes[1, 0].legend(title="")

#     # Plot 4: Summary statistics
#     stats_data = []
#     for optimizer, deltas in all_deltas.items():
#         deltas = np.array(deltas)

#         if len(deltas) > 0:
#             stats_data.append(
#                 {"optimizer": optimizer, "metric": "Max Improvement", "value": np.max(deltas)}
#             )
#             stats_data.append(
#                 {
#                     "optimizer": optimizer,
#                     "metric": "Avg Improvement",
#                     "value": np.mean(deltas[deltas > 0]) if any(deltas > 0) else 0,
#                 }
#             )
#             stats_data.append(
#                 {"optimizer": optimizer, "metric": "Avg Change", "value": np.mean(deltas)}
#             )
#             stats_data.append(
#                 {"optimizer": optimizer, "metric": "Median Change", "value": np.median(deltas)}
#             )
#             stats_data.append(
#                 {"optimizer": optimizer, "metric": "Volatility", "value": np.std(deltas)}
#             )

#     stats_df = pd.DataFrame(stats_data)
#     sns.barplot(x="optimizer", y="value", hue="metric", data=stats_df, ax=axes[1, 1])
#     axes[1, 1].set_title("Optimization Strategy Metrics")
#     axes[1, 1].set_xlabel("Optimizer")
#     axes[1, 1].set_ylabel("Value")
#     axes[1, 1].legend(title="", loc="upper left", bbox_to_anchor=(1, 1))

#     plt.tight_layout()
#     plt.suptitle(f"Optimization Strategy Comparison: {dataset} - {model}", y=1.02, fontsize=16)

#     return fig, all_deltas

In [ ]:
# # run
# analyze_prompt_diversity("gsm8k", "llama", ["CAPO", "EvoPromptGA"], max_steps=100)